<a href="https://colab.research.google.com/github/arthcras/bitcoin/blob/main/KUCOINmet_saldo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ccxt

In [ ]:
pip install neo4j

In [ ]:
import ccxt
from neo4j import GraphDatabase
import csv
from datetime import datetime

# ===== CONFIG =====
START_AMOUNT = 1000
FEE_RATE = 0.001  # 0.1%
MIN_PROFIT_THRESHOLD = 1.0  # minimale winst in USDT

# === Bestandsnamen ===
CSV_FILE = "arbitrage_trades_kucoin.csv"
WALLET_CSV = "wallet_states_kucoin.csv"
SUMMARY_CSV = "path_profits_kucoin.csv"
LOSSES_CSV = "loss_paths_kucoin.csv"

# === Neo4j Connectie ===
driver = GraphDatabase.driver(
    "neo4j+s://50e6487d.databases.neo4j.io",
    auth=("neo4j", "YYPOyOPLlIAaeRIH5TfOFpnspjtL2LV7VgyQsf4Xwsc")
)

# === KuCoin Exchange Setup ===
kucoin = ccxt.kucoin({
    'apiKey': '68536e9b610c550001cec5f3',
    'secret': '72fcdb78-4b20-4165-a288-f12ddcddd0cf',
    'password': '789NDTHD8',
})
kucoin.load_markets()

# === Hulpfunctie wallet-logging ===
def log_wallet_state(wallet, label, writer):
    timestamp = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    for coin in sorted(wallet.keys()):
        writer.writerow([timestamp, label, coin, wallet[coin]])

# === Haal arbitragepaden op uit Neo4j ===
def fetch_arbitrage_paths():
    query = """
    WITH 1.0 AS startVal, ["KuCoin"] AS allowedMarkets
    MATCH p = (c:Currency)-[rels:EXCHANGE*3]->(c_end:Currency)
    WHERE c.name = "USDT" AND c_end.name = "USDT"
      AND ALL(rel IN rels WHERE rel.market IN allowedMarkets AND rel.rate > 0)
    WITH p, rels AS relationships, NODES(p) AS nodes,
         [rel IN rels | coalesce(rel.spread, 0.01)] AS spreads, startVal
    WITH p, relationships, nodes, spreads, startVal,
         REDUCE(s = startVal, i IN RANGE(0, SIZE(relationships)-1) |
            s * relationships[i].rate * (1 - spreads[i]/100.0)
         ) AS endVal
    RETURN [n IN nodes | n.name] AS CurrencyPath,
           [r IN relationships | r.market] AS ExchangeNames,
           [r IN relationships | r.rate] AS ConversionRates,
           spreads AS Spreads,
           endVal - startVal AS Profit
    ORDER BY Profit DESC LIMIT 1000;
    """
    with driver.session() as session:
        result = session.run(query)
        return [r.data() for r in result]

# === CSV-bestanden openen ===
trade_csv = open(CSV_FILE, "w", newline="")
trade_writer = csv.writer(trade_csv)
trade_writer.writerow([
    "timestamp", "path_index", "step", "from", "to", "exchange",
    "rate", "spread", "amount_before", "amount_after", "symbol", "side"
])

wallet_csv = open(WALLET_CSV, "w", newline="")
wallet_writer = csv.writer(wallet_csv)
wallet_writer.writerow(["timestamp", "label", "coin", "balance"])

summary_csv = open(SUMMARY_CSV, "w", newline="")
summary_writer = csv.writer(summary_csv)
summary_writer.writerow(["timestamp", "path_index", "profit_usdt", "final_balance_usdt"])

losses_csv = open(LOSSES_CSV, "w", newline="")
losses_writer = csv.writer(losses_csv)
losses_writer.writerow(["timestamp", "path_index", "loss_usdt", "final_balance_usdt"])

# === Main executielus ===
paths = fetch_arbitrage_paths()
total_profit = 0

for idx, record in enumerate(paths):
    path = record["CurrencyPath"]
    exchanges = record["ExchangeNames"]
    rates = record["ConversionRates"]
    spreads = record["Spreads"]

    print(f"\n🔁 Pad {idx+1}: {' -> '.join(path)} via {exchanges}")

    wallet = {coin: 0.0 for coin in set(path)}
    wallet[path[0]] = START_AMOUNT
    log_wallet_state(wallet, f"path_{idx+1}_start", wallet_writer)

    for i in range(len(path) - 1):
        from_coin = path[i]
        to_coin = path[i+1]
        exchange = kucoin
        rate = rates[i]
        spread = spreads[i]
        fee_mult = 1 - FEE_RATE
        spread_mult = 1 - spread / 100

        amount_before = wallet[from_coin]
        amount_after = amount_before * rate * spread_mult * fee_mult

        wallet[from_coin] = 0
        wallet[to_coin] += amount_after

        symbol_dir = f"{from_coin}/{to_coin}"
        symbol_inv = f"{to_coin}/{from_coin}"
        if symbol_dir in exchange.symbols:
            symbol = symbol_dir
            side = "sell"
        elif symbol_inv in exchange.symbols:
            symbol = symbol_inv
            side = "buy"
        else:
            print(f"❌ Geen pair {from_coin}/{to_coin} op KuCoin")
            break

        ts = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
        trade_writer.writerow([
            ts, idx+1, f"step_{i+1}",
            from_coin, to_coin, "KuCoin",
            rate, spread, amount_before, amount_after, symbol, side
        ])
        log_wallet_state(wallet, f"path_{idx+1}_hop_{i+1}", wallet_writer)

        print(f"Step {i+1}: {from_coin} → {to_coin} | "
              f"{amount_before:.6f} → {amount_after:.6f}")

    final_balance = wallet.get("USDT", 0)
    profit = final_balance - START_AMOUNT
    total_profit += profit

    log_wallet_state(wallet, f"path_{idx+1}_end", wallet_writer)

    timestamp = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    profit_rounded = round(profit, 6)
    final_rounded = round(final_balance, 6)

    if profit_rounded < 0 or profit_rounded < MIN_PROFIT_THRESHOLD:
        losses_writer.writerow([timestamp, idx+1, profit_rounded, final_rounded])
    else:
        summary_writer.writerow([timestamp, idx+1, profit_rounded, final_rounded])

# === Totale winst loggen ===
summary_writer.writerow(["", "TOTAAL", round(total_profit, 6), ""])

# === Sluit alles af ===
trade_csv.close()
wallet_csv.close()
summary_csv.close()
losses_csv.close()
driver.close()

print("\n✅ Script succesvol uitgevoerd voor KuCoin. Resultaten opgeslagen.")


ExchangeError: kucoin {"msg":"Our services are currently unavailable in the U.S. To ensure a seamless experience, please access the platform from a non-restricted country/region using a supported IP address. For more details, please refer to our Terms of Use. (current ip: 35.243.132.127 and current area: US)","code":"400302"}